# Azure Cognitive Search Vector Search Code Sample with Azure OpenAI
This code demonstrates how to use Azure Cognitive Search with OpenAI and Azure Python SDK
## Prerequisites
To run the code, install the following packages. Please use the latest pre-release version `pip install azure-search-documents --pre`.

In [ ]:
! pip install azure-search-documents --pre
! pip install openai
! pip install python-dotenv

## Import required libraries and environment variables

In [1]:
# Import required libraries  
import os  
import json  
import openai  
from dotenv import load_dotenv  
from pathlib import Path  
from tenacity import retry, wait_random_exponential, stop_after_attempt  
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient  
from azure.search.documents.indexes import SearchIndexClient  
from azure.search.documents.models import Vector  
from azure.search.documents.indexes.models import (  
    SearchIndex,  
    SearchField,  
    SearchFieldDataType,  
    SimpleField,  
    SearchableField,  
    SearchIndex,  
    SemanticConfiguration,  
    PrioritizedFields,  
    SemanticField,  
    SearchField,  
    SemanticSettings,  
    VectorSearch,  
    HnswVectorSearchAlgorithmConfiguration,  
)  
  
# Configure environment variables  
env_path = Path('.') / 'secrets.env'
load_dotenv(dotenv_path=env_path)
service_endpoint = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT") 
index_name = os.getenv("AZURE_SEARCH_INDEX_NAME") 
key = os.getenv("AZURE_SEARCH_ADMIN_KEY") 
openai.api_type = "azure"  
openai.api_key = os.getenv("AZURE_OPENAI_API_KEY")  
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")  
openai.api_version = os.getenv("AZURE_OPENAI_API_VERSION")  
credential = AzureKeyCredential(key)
cache_index_name = os.getenv("CACHE_INDEX_NAME")

## Create embeddings
Read your data, generate OpenAI embeddings and export to a format to insert your Azure Cognitive Search index:

In [14]:
# Generate Document Embeddings using OpenAI Ada 002

# Read the text-sample.json
with open('data/articles.json', 'r', encoding='utf-8') as file:
    input_data = json.load(file)

@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
# Function to generate embeddings for title and content fields, also used for query embeddings
def generate_embeddings(text):
    response = openai.Embedding.create(
        input=text, engine="text-embedding-ada-002")
    embeddings = response['data'][0]['embedding']
    return embeddings

output=[]
# Generate embeddings for title and content fields
for item in input_data.items():
    output_item={}
    title = item[0]
    content = item[1]
    content_embeddings = generate_embeddings(content)
    output_item['id'] = title
    output_item['content'] = content
    output_item['contentVector'] = content_embeddings
    output.append(output_item)


# Output embeddings to docVectors.json file
with open("data/docVectors.json", "w") as f:
    json.dump(output, f)

## Create your search index
Create your search index schema and vector search configuration:

In [ ]:
# Create a search index
index_client = SearchIndexClient(
    endpoint=service_endpoint, credential=credential)
fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True, sortable=True, filterable=True, facetable=True),
    SearchableField(name="content", type=SearchFieldDataType.String),
    SearchField(name="contentVector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                searchable=True, vector_search_dimensions=1536, vector_search_configuration="my-vector-config"),
]

vector_search = VectorSearch(
    algorithm_configurations=[
        HnswVectorSearchAlgorithmConfiguration(
            name="my-vector-config",
            kind="hnsw",
            parameters={
                "m": 4,
                "efConstruction": 400,
                "efSearch": 500,
                "metric": "cosine"
            }
        )
    ]
)

semantic_config = SemanticConfiguration(
    name="my-semantic-config",
    prioritized_fields=PrioritizedFields(
        title_field=SemanticField(field_name="id"),
        prioritized_content_fields=[SemanticField(field_name="content")]
    )
)

# Create the semantic settings with the configuration
semantic_settings = SemanticSettings(configurations=[semantic_config])

# Create the search index with the semantic settings
index = SearchIndex(name=index_name, fields=fields,
                    vector_search=vector_search, semantic_settings=semantic_settings)
result = index_client.create_or_update_index(index)
print(f' {result.name} created')


### Create Semantic Cache Index

In [3]:
# Create a search index

index_client = SearchIndexClient(
    endpoint=service_endpoint, credential=credential)
fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True, sortable=True, filterable=True, facetable=True),
    SearchableField(name="search_query", type=SearchFieldDataType.String),
    SearchField(name="search_query_vector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                searchable=True, vector_search_dimensions=1536, vector_search_configuration="my-vector-config"),
    SearchableField(name="gpt_response", type=SearchFieldDataType.String),

]

vector_search = VectorSearch(
    algorithm_configurations=[
        HnswVectorSearchAlgorithmConfiguration(
            name="my-vector-config",
            kind="hnsw",
            parameters={
                "m": 4,
                "efConstruction": 400,
                "efSearch": 500,
                "metric": "cosine"
            }
        )
    ]
)


# Create the search index with the semantic settings
index = SearchIndex(name=cache_index_name, fields=fields,
                    vector_search=vector_search)
result = index_client.create_or_update_index(index)
print(f' {result.name} created')


 payroll_hr_cache created


## Insert text and embeddings into vector store
Add texts and metadata from the JSON data to the vector store:

In [17]:
# Upload some documents to the index
with open('data/docVectors.json', 'r') as file:  
    documents = json.load(file)  
search_client = SearchClient(endpoint=service_endpoint, index_name=index_name, credential=credential)
result = search_client.upload_documents(documents)  
print(f"Uploaded {len(documents)} documents") 

Uploaded 100 documents


## Perform a vector similarity search

In [18]:
# Pure Vector Search
query = "time to receive w2 form"  
  
search_client = SearchClient(service_endpoint, index_name, credential=credential)
vector = Vector(value=generate_embeddings(query), k=3, fields="contentVector")
  
results = search_client.search(  
    search_text=None,  
    vectors= [vector],
    select=["id", "content"],
)  
  
for result in results:  
    print(f"Title: {result['id']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['content']}")  


Title: article_3
Score: 0.8511498
Content: 4. Understanding Your W-2: A Guide for ADP Payroll Customers 

If you are an employee whose payroll is managed by ADP, you will receive a W-2 form at the end of each fiscal year. This form summarizes your earnings and taxes paid throughout the year, and is necessary for filing your tax returns.

To help you understand the information displayed on your W-2 form, we've created this guide:

1. Box 1 - Wages, tips, and other compensation: This box includes all taxable income earned during the year, including bonuses, overtime pay, and vacation pay.

2. Box 2 - Federal income tax withheld: This box shows the total amount of federal income tax withheld from your paycheck throughout the year. This amount is based on your W-4 form, which you fill out when you are hired.

3. Box 3 - Social Security wages: This box shows the total amount of wages that are subject to Social Security taxes. This includes all taxable income up to the Social Security wage b

In [20]:
# Pure Vector Search multi-lingual (e.g 'tools for software development' in Dutch)  
query = "Zeit, das W2-Formular zu erhalten"  
  
search_client = SearchClient(service_endpoint, index_name, credential=credential)
vector = Vector(value=generate_embeddings(query), k=3, fields="contentVector")  
  
results = search_client.search(  
    search_text=None,  
    vectors=[vector],
    select=["id", "content"],
)  
  
for result in results:  
    print(f"Title: {result['id']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['content']}")  


Title: article_3
Score: 0.8379801
Content: 4. Understanding Your W-2: A Guide for ADP Payroll Customers 

If you are an employee whose payroll is managed by ADP, you will receive a W-2 form at the end of each fiscal year. This form summarizes your earnings and taxes paid throughout the year, and is necessary for filing your tax returns.

To help you understand the information displayed on your W-2 form, we've created this guide:

1. Box 1 - Wages, tips, and other compensation: This box includes all taxable income earned during the year, including bonuses, overtime pay, and vacation pay.

2. Box 2 - Federal income tax withheld: This box shows the total amount of federal income tax withheld from your paycheck throughout the year. This amount is based on your W-4 form, which you fill out when you are hired.

3. Box 3 - Social Security wages: This box shows the total amount of wages that are subject to Social Security taxes. This includes all taxable income up to the Social Security wage b

## Perform a Hybrid Vector Search

In [22]:
# Cross-Field Vector Search
query = "time to receive w2 form"  
  
search_client = SearchClient(service_endpoint, index_name, credential=credential)  
vector = Vector(value=generate_embeddings(query), k=3, fields="contentVector")  
  
results = search_client.search(  
    search_text=query,  
    vectors=[vector],
    filter="id eq 'article_3'",
    select=["id", "content"],
)  
  
for result in results:  
    print(f"Title: {result['id']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['content']}")  


Title: article_3
Score: 0.03333333507180214
Content: 4. Understanding Your W-2: A Guide for ADP Payroll Customers 

If you are an employee whose payroll is managed by ADP, you will receive a W-2 form at the end of each fiscal year. This form summarizes your earnings and taxes paid throughout the year, and is necessary for filing your tax returns.

To help you understand the information displayed on your W-2 form, we've created this guide:

1. Box 1 - Wages, tips, and other compensation: This box includes all taxable income earned during the year, including bonuses, overtime pay, and vacation pay.

2. Box 2 - Federal income tax withheld: This box shows the total amount of federal income tax withheld from your paycheck throughout the year. This amount is based on your W-4 form, which you fill out when you are hired.

3. Box 3 - Social Security wages: This box shows the total amount of wages that are subject to Social Security taxes. This includes all taxable income up to the Social Secur